# Google Colab Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/DataScience/machine_learning/kaggle/Titanic

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/DataScience/machine_learning/kaggle/Titanic


In [3]:
import os
# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Libraries

In [4]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# Data scaling
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Evaluation
from sklearn.metrics import mean_squared_error

In [5]:
# Libraries for custom Transformer
from sklearn.base import BaseEstimator, TransformerMixin

# Data Loading

In [88]:
import pandas as pd
DATA_ROOT = 'datasets'

train = pd.read_csv(os.path.join(DATA_ROOT, 'train.csv'))
test = pd.read_csv(os.path.join(DATA_ROOT, 'test.csv'))

In [89]:
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [90]:
train.shape

(891, 12)

In [91]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [92]:
train = train.set_index('PassengerId')
test = test.set_index('PassengerId')

In [204]:
test.index

Int64Index([ 892,  893,  894,  895,  896,  897,  898,  899,  900,  901,
            ...
            1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309],
           dtype='int64', name='PassengerId', length=418)

# Data Type

In [93]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [94]:
num_cols = [col for col in train.columns if train[col].dtype in ['int64', 'float64']]
cat_cols = [col for col in train.columns if train[col].dtype in ['object']]

print('num_cols:', num_cols)
print('cat_cols:', cat_cols)

num_cols: ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
cat_cols: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


In [95]:
card = [train[col].nunique() for col in cat_cols]
for a, b in zip(cat_cols, card):
    print(a, b)

low_card_cols = [col for col in cat_cols if train[col].nunique() <= 10]
high_card_cols = [col for col in cat_cols if train[col].nunique() > 10]
print('low_card_cols:', low_card_cols)
print('high_card_cols:', high_card_cols)

Name 891
Sex 2
Ticket 681
Cabin 147
Embarked 3
low_card_cols: ['Sex', 'Embarked']
high_card_cols: ['Name', 'Ticket', 'Cabin']


# Data Cleaning

In [96]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [97]:
# Inpute Age column with median age of the corresponding class
pclass = train.Pclass.unique()
median_age_pclass = [train[train['Pclass'] == c]['Age'].median() for c in pclass]
print('median_age_pclass:', median_age_pclass)

# TODO: finish custom class for imputing median_age_pclass

median_age_pclass: [24.0, 37.0, 29.0]


## Filling NaN Values

In [145]:
imputer = ColumnTransformer([
    ('imputer_num', SimpleImputer(strategy='median'), num_cols),
    ('imputer_cat', SimpleImputer(strategy='most_frequent'), cat_cols)
])

In [146]:
clean_pipe = Pipeline([
    ('imputer', imputer)
])

In [102]:
train_clean = imputer.fit_transform(train)
train_clean

array([[0.0, 3.0, 22.0, ..., 'A/5 21171', 'B96 B98', 'S'],
       [1.0, 1.0, 38.0, ..., 'PC 17599', 'C85', 'C'],
       [1.0, 3.0, 26.0, ..., 'STON/O2. 3101282', 'B96 B98', 'S'],
       ...,
       [0.0, 3.0, 28.0, ..., 'W./C. 6607', 'B96 B98', 'S'],
       [1.0, 1.0, 26.0, ..., '111369', 'C148', 'C'],
       [0.0, 3.0, 32.0, ..., '370376', 'B96 B98', 'Q']], dtype=object)

In [103]:
train_clean = pd.DataFrame(train_clean)
train_clean.columns = num_cols + cat_cols
train_clean.set_index(train.index)
train_clean.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Cabin,Embarked
0,0.0,3.0,22.0,1.0,0.0,7.25,"Braund, Mr. Owen Harris",male,A/5 21171,B96 B98,S
1,1.0,1.0,38.0,1.0,0.0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,1.0,3.0,26.0,0.0,0.0,7.925,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,B96 B98,S
3,1.0,1.0,35.0,1.0,0.0,53.1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,0.0,3.0,35.0,0.0,0.0,8.05,"Allen, Mr. William Henry",male,373450,B96 B98,S


In [104]:
train_clean.shape

(891, 11)

In [105]:
# After imputing, all dtype will change to 'object'
# So need to reassign float64 type to num_cols
X_ = train_clean[num_cols].astype('float64')
train_clean = X_.join(train_clean[cat_cols])

In [106]:
train_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    float64
 1   Pclass    891 non-null    float64
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    float64
 4   Parch     891 non-null    float64
 5   Fare      891 non-null    float64
 6   Name      891 non-null    object 
 7   Sex       891 non-null    object 
 8   Ticket    891 non-null    object 
 9   Cabin     891 non-null    object 
 10  Embarked  891 non-null    object 
dtypes: float64(6), object(5)
memory usage: 76.7+ KB


# Data Analysis

In [108]:
train_clean.head(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Cabin,Embarked
0,0.0,3.0,22.0,1.0,0.0,7.2500,"Braund, Mr. Owen Harris",male,A/5 21171,B96 B98,S
1,1.0,1.0,38.0,1.0,0.0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C


In [109]:
train_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    float64
 1   Pclass    891 non-null    float64
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    float64
 4   Parch     891 non-null    float64
 5   Fare      891 non-null    float64
 6   Name      891 non-null    object 
 7   Sex       891 non-null    object 
 8   Ticket    891 non-null    object 
 9   Cabin     891 non-null    object 
 10  Embarked  891 non-null    object 
dtypes: float64(6), object(5)
memory usage: 76.7+ KB


## Age Bucket
Divide ages into several intervals, that each age interval will have different survival rate

In [110]:
train_clean['AgeBucket'] = train_clean['Age'] // 15 * 15
train_clean[['AgeBucket', 'Survived']].groupby(['AgeBucket']).mean()

,Survived
AgeBucket,
0.0,0.576923
15.0,0.337474
30.0,0.423256
45.0,0.404494
60.0,0.240000
75.0,1.000000


## Relatives On Board

In [112]:
train_clean['RelativesOnboard'] = train_clean.SibSp + train_clean.Parch
train_clean[['RelativesOnboard', 'Survived']].groupby(['RelativesOnboard']).mean()

,Survived
RelativesOnboard,
0.0,0.303538
1.0,0.552795
2.0,0.578431
3.0,0.724138
4.0,0.200000
5.0,0.136364
6.0,0.333333
7.0,0.000000
10.0,0.000000


## Name Title

In [115]:
train_clean.Name.head(20)

0                               Braund, Mr. Owen Harris
1     Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                Heikkinen, Miss. Laina
3          Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                              Allen, Mr. William Henry
5                                      Moran, Mr. James
6                               McCarthy, Mr. Timothy J
7                        Palsson, Master. Gosta Leonard
8     Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
9                   Nasser, Mrs. Nicholas (Adele Achem)
10                      Sandstrom, Miss. Marguerite Rut
11                             Bonnell, Miss. Elizabeth
12                       Saundercock, Mr. William Henry
13                          Andersson, Mr. Anders Johan
14                 Vestrom, Miss. Hulda Amanda Adolfina
15                     Hewlett, Mrs. (Mary D Kingcome) 
16                                 Rice, Master. Eugene
17                         Williams, Mr. Charles

In [127]:
def extract(name, titles):
    for t in titles:
        if t in name:
            return t
    return np.nan

title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer']

train_clean['Title'] = train_clean['Name'].map(lambda name: extract(name, title_list))


In [125]:
for a, b in zip(train_clean.Title.head(10), train_clean.Name.head(10)):
    print(a, ',', b)


Mr , Braund, Mr. Owen Harris
Mrs , Cumings, Mrs. John Bradley (Florence Briggs Thayer)
Miss , Heikkinen, Miss. Laina
Mrs , Futrelle, Mrs. Jacques Heath (Lily May Peel)
Mr , Allen, Mr. William Henry
Mr , Moran, Mr. James
Mr , McCarthy, Mr. Timothy J
Master , Palsson, Master. Gosta Leonard
Mrs , Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
Mrs , Nasser, Mrs. Nicholas (Adele Achem)


In [122]:
train_clean[train_clean.Title.isna()]
# Or: # train_clean[train_clean.Title.isnull()]

,Survived,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Cabin,Embarked,AgeBucket,RelativesOnboard,Title


In [123]:
train_clean.Title.value_counts()

Mr          518
Miss        180
Mrs         129
Master       40
Dr            7
Rev           6
Major         2
Col           2
Don           1
Mme           1
Ms            1
Mlle          1
Capt          1
Countess      1
Jonkheer      1
Name: Title, dtype: int64

## Cabin to Deck 

In [131]:
train_clean.Cabin.head(10)

0    B96 B98
1        C85
2    B96 B98
3       C123
4    B96 B98
5    B96 B98
6        E46
7    B96 B98
8    B96 B98
9    B96 B98
Name: Cabin, dtype: object

In [128]:
#Turning cabin number into Deck
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
train_clean['Deck'] = train_clean['Cabin'].map(lambda x: extract(x, cabin_list))

In [129]:
for a, b in zip(train_clean.Deck.head(10), train_clean.Cabin.head(10)):
    print(a, ',', b)

B , B96 B98
C , C85
B , B96 B98
C , C123
B , B96 B98
B , B96 B98
E , E46
B , B96 B98
B , B96 B98
B , B96 B98


# Data Split

In [132]:
# Data split

y_train = train_clean['Survived']
X_train = train_clean.drop(['Survived'], axis=1)

# Pipleline

In [134]:
X_train.dtypes

Pclass              float64
Age                 float64
SibSp               float64
Parch               float64
Fare                float64
Name                 object
Sex                  object
Ticket               object
Cabin                object
Embarked             object
AgeBucket           float64
RelativesOnboard    float64
Title                object
Deck                 object
dtype: object

In [135]:
num_cols = ['Pclass', 'SibSp', 'Parch', 'Fare', 'AgeBucket', 'RelativesOnboard']
cat_cols = ['Sex', 'Embarked', 'Title', 'Deck']

In [137]:
card = [X_train[col].nunique() for col in cat_cols]
for a, b in zip(cat_cols, card):
    print(a, b)

low_card_cols = [col for col in cat_cols if X_train[col].nunique() <= 10]
high_card_cols = [col for col in cat_cols if X_train[col].nunique() > 10]
print('low_card_cols:', low_card_cols)
print('high_card_cols:', high_card_cols)

Sex 2
Embarked 3
Title 15
Deck 8
low_card_cols: ['Sex', 'Embarked', 'Deck']
high_card_cols: ['Title']


### High Card Trans

In [139]:
# High Cardinality Transformer: perform one-hot transformer for high-cardinality
# columns while avoiding generating too much dummpy features. It will only one-hot
# the classes with high proportion. A threshold can be set.

from collections import Counter

def cumulatively_categorise(column,threshold=0.75,return_categories_list=True):
    #Find the threshold value using the percentage and number of instances in the column
    threshold_value=int(threshold*len(column))
    #Initialise an empty list for our new minimised categories
    categories_list=[]
    #Initialise a variable to calculate the sum of frequencies
    s=0
    #Create a counter dictionary of the form unique_value: frequency
    counts=Counter(column)

    #Loop through the category name and its corresponding frequency after sorting the categories by descending order of frequency
    for i,j in counts.most_common():
        #Add the frequency to the global sum
        s+=dict(counts)[i]
        #Append the category name to the list
        categories_list.append(i)
        #Check if the global sum has reached the threshold value, if so break the loop
        if s>=threshold_value:
            break
    #Append the category Other to the list
    categories_list.append('Other')

    #Replace all instances not in our new categories by Other  
    new_column=column.apply(lambda x: x if x in categories_list else 'Other')

    #Return transformed column and unique values if return_categories=True
    if(return_categories_list):
        return new_column,categories_list
    #Return only the transformed column if return_categories=False
    else:
        return new_column

In [140]:
# high_card_cols = ['Regionname', 'CouncilArea']
class HighCardAggregation(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.cols = high_card_cols
        pass

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        # X is a numpy array, the columns to with high cardinality
        X_ = X.copy()
        X_df = pd.DataFrame(X, columns=high_card_cols)
        
        for col in self.cols:
            # transformed_columns is a pandas dataframe
            transformed_column, trans_list = cumulatively_categorise(column=X_df[col], threshold=0.75)
#             print('transformed_column: \n', transformed_column.value_counts())
            X_ = np.c_[X_, transformed_column.to_numpy()]
            
        X_ = np.delete(X_, [i for i in range(len(self.cols))], 1)
        return X_

## Pipelines

In [167]:
num_cols = ['Pclass', 'SibSp', 'Parch', 'Fare', 'AgeBucket', 'RelativesOnboard']
cat_cols = ['Sex', 'Embarked', 'Title', 'Deck']
low_card_cols = ['Sex', 'Embarked', 'Deck']
high_card_cols = ['Title']

low_card_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

high_card_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('aggregation', HighCardAggregation()),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

# Numerical transformer
num_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

# Categorical transformers
cat_trans = ColumnTransformer(transformers=[
    ('low_card_pipeline', low_card_pipe, low_card_cols),
    ('high_card_pipeline', high_card_pipe, high_card_cols)
])

imputer = ColumnTransformer([
    ('imputer_num', SimpleImputer(strategy='median'), num_cols),
    ('imputer_cat', SimpleImputer(strategy='most_frequent'), cat_cols)
])

clean_pipe = Pipeline([
    ('imputer', imputer)
])

num_cat_col_trans = ColumnTransformer(transformers=[
    ('num', num_trans, num_cols),
    ('cat', cat_trans, cat_cols)
])

preprocessor = Pipeline([
    # ('cleaning', clean_pipe),
    ('num_cat', num_cat_col_trans)
])

In [177]:
print(num_cols)
print(cat_cols)
print(low_card_cols)
print(high_card_cols)

['Pclass', 'SibSp', 'Parch', 'Fare', 'AgeBucket', 'RelativesOnboard']
['Sex', 'Embarked', 'Title', 'Deck']
['Sex', 'Embarked', 'Deck']
['Title']


# Train

In [178]:
X_train_prepared = preprocessor.fit_transform(X_train[num_cols + cat_cols])

## Model

In [179]:
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
# forest_reg.fit(X_train_prepared, y_train)

# Test

In [163]:
num_cols = [col for col in test.columns if test[col].dtype in ['int64', 'float64']]
cat_cols = [col for col in test.columns if test[col].dtype in ['object']]

imputer = ColumnTransformer([
    ('imputer_num', SimpleImputer(strategy='median'), num_cols),
    ('imputer_cat', SimpleImputer(strategy='most_frequent'), cat_cols)
])

test_clean = imputer.fit_transform(test)

test_clean = pd.DataFrame(test_clean)
test_clean.columns = num_cols + cat_cols
test_clean.set_index(test.index)
test_clean.head()

# After imputing, all dtype will change to 'object'
# So need to reassign float64 type to num_cols
X_ = test_clean[num_cols].astype('float64')
test_clean = X_.join(test_clean[cat_cols])

In [165]:
test_clean['AgeBucket'] = test_clean['Age'] // 15 * 15
test_clean['RelativesOnboard'] = test_clean.SibSp + test_clean.Parch
test_clean['Title'] = test_clean['Name'].map(lambda name: extract(name, title_list))
test_clean['Deck'] = test_clean['Cabin'].map(lambda x: extract(x, cabin_list))

In [166]:
test_clean.head()

,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Cabin,Embarked,AgeBucket,RelativesOnboard,Title,Deck
0,3.0,34.5,0.0,0.0,7.8292,"Kelly, Mr. James",male,330911,B57 B59 B63 B66,Q,30.0,0.0,Mr,B
1,3.0,47.0,1.0,0.0,7.0000,"Wilkes, Mrs. James (Ellen Needs)",female,363272,B57 B59 B63 B66,S,45.0,1.0,Mrs,B
2,2.0,62.0,0.0,0.0,9.6875,"Myles, Mr. Thomas Francis",male,240276,B57 B59 B63 B66,Q,60.0,0.0,Mr,B
3,3.0,27.0,0.0,0.0,8.6625,"Wirz, Mr. Albert",male,315154,B57 B59 B63 B66,S,15.0,0.0,Mr,B
4,3.0,22.0,1.0,1.0,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,3101298,B57 B59 B63 B66,S,15.0,2.0,Mrs,B


In [173]:
X_test_prepared = preprocessor.transform(test_clean[num_cols + cat_cols])
y_pred = forest_reg.predict(X_test_prepared)

In [182]:
print(type(y_pred))

<class 'numpy.ndarray'>


In [231]:
y_pred_class = [0 if pred < 0.5 else 1 for pred in y_pred]

In [232]:
y_pred_class = pd.DataFrame(data={'Survived': y_pred_class}, index=test.index)

In [234]:
y_pred_class.to_csv(os.path.join(DATA_ROOT, 'predictions.csv'))

In [176]:
X_train

,Pclass,Age,SibSp,Parch,Fare,Name,Sex,Ticket,Cabin,Embarked,AgeBucket,RelativesOnboard,Title,Deck
0,3.0,22.0,1.0,0.0,7.2500,"Braund, Mr. Owen Harris",male,A/5 21171,B96 B98,S,15.0,1.0,Mr,B
1,1.0,38.0,1.0,0.0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,30.0,1.0,Mrs,C
2,3.0,26.0,0.0,0.0,7.9250,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,B96 B98,S,15.0,0.0,Miss,B
3,1.0,35.0,1.0,0.0,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,30.0,1.0,Mrs,C
4,3.0,35.0,0.0,0.0,8.0500,"Allen, Mr. William Henry",male,373450,B96 B98,S,30.0,0.0,Mr,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2.0,27.0,0.0,0.0,13.0000,"Montvila, Rev. Juozas",male,211536,B96 B98,S,15.0,0.0,Rev,B
887,1.0,19.0,0.0,0.0,30.0000,"Graham, Miss. Margaret Edith",female,112053,B42,S,15.0,0.0,Miss,B
888,3.0,28.0,1.0,2.0,23.4500,"Johnston, Miss. Catherine Helen ""Carrie""",female,W./C. 6607,B96 B98,S,15.0,3.0,Miss,B
889,1.0,26.0,0.0,0.0,30.0000,"Behr, Mr. Karl Howell",male,111369,C148,C,15.0,0.0,Mr,C


In [181]:
from sklearn.model_selection import cross_val_score
forest_scores = cross_val_score(forest_reg, X_train[num_cols + cat_cols], y_train, cv=10)
forest_scores.mean()

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 328, in fit
    X, y, multi_output=True, accept_sparse="csc", dtype=DTYPE
  File "/usr/local/lib/python3.7/dist-packages/sklearn/base.py", line 581, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/usr/local/lib/pyth

nan